## Get data

In [ ]:
!pip install hackernews-client transformers kaggle catalyst  -q 

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /content

In [ ]:
!kaggle datasets download -d rmisra/news-category-dataset > "error"
!cp '/gdrive/My Drive/kaggle.json' /root/.kaggle/
!kaggle datasets download -d rmisra/news-category-dataset
!git clone https://github.com/huggingface/transformers;cd transformers; pip install -q .
!unzip news-category-dataset.zip
!pip install -q catalyst
!git clone https://github.com/mhjabreel/CharCnn_Keras
!wget https://www.cs.umb.edu/~smimarog/textmining/datasets/r8-train-all-terms.txt
!wget https://www.cs.umb.edu/~smimarog/textmining/datasets/r8-test-all-terms.txt
!wget https://www.cs.umb.edu/~smimarog/textmining/datasets/r52-train-all-terms.txt
!wget https://www.cs.umb.edu/~smimarog/textmining/datasets/r52-test-all-terms.txt


In [ ]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset,DataLoader
import requests

from datetime import datetime, timedelta

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import f1_score, classification_report

# Core
import pandas as pd
import numpy as np
# Data Visualization
import seaborn as sns
sns.set_style("darkgrid")
import matplotlib.pyplot as plt
# SKlearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, f1_score
# PyTorch 
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
# Transformers 
from transformers import AutoConfig, AutoModel, AutoTokenizer
# Catalyst
from catalyst.dl import SupervisedRunner
from catalyst.dl.callbacks import AccuracyCallback, F1ScoreCallback, OptimizerCallback
from catalyst.dl.callbacks import CheckpointCallback, InferCallback
from catalyst.utils import set_global_seed, prepare_cudnn
# Extras
import warnings
warnings.filterwarnings('ignore')
import os
import logging
from typing import Mapping, List
import datetime as dt


from IPython.display import HTML, Image

In [ ]:
d0 = pd.read_json("News_Category_Dataset_v2.json",lines=True)
df = d0[['category']]
df['text'] = (d0['headline'] + " " + d0['short_description']).str.lower()

reduced_cats = {"CULTURE & ARTS":["ARTS","ARTS & CULTURE","CULTURE & ARTS"],
 "BUSINESS": ["BUSINESS"],
 "MONEY":["MONEY"],
 "EDUCATION": ["EDUCATION","COLLEGE"],
 "COMEDY & ENTERTAINMENT" : ["COMEDY","ENTERTAINMENT","MEDIA"],
 "FAMILY" : ["PARENTING","DIVORCE","WEDDINGS"],
 "OPINION": ["BLACK VOICES","LATINO VOICES","QUEER VOICES","WOMEN"],
 "FOOD & TASTE" : ["FOOD & DRINK","TASTE"],
 "HEALTH & LIVING": ["WELLNESS","HEALTHY LIVING","STYLE & BEAUTY","HOME & LIVING","STYLE"],
 "RELIGION" : ["RELIGION"],
 "POLITICS" : ["POLITICS"],
 "SPORTS" : ["SPORTS"],
 "TRAVEL" : ["TRAVEL"],
 "NEWS" :["GOOD NEWS","WORLDPOST","WORLD NEWS","WEIRD NEWS"],
 "ENVIRONMENT" : ["GREEN","ENVIRONMENT"],
 "CRIME" :["CRIME"],
 "SCIENCE": ["SCIENCE"],
 "TECH": ["TECH"]
 }
flat_list = [item for sublist in reduced_cats.values() for item in sublist]
df = df[df['category'].isin(flat_list)]
df['category'] = df['category'].map({item:cat for cat in reduced_cats for item in reduced_cats[cat]})

In [ ]:
#https://github.com/arimbr/valohai-fasttext-example
REDDIT_S3_PATH = 'https://valohai-fasttext-example.s3.eu-west-3.amazonaws.com/reddit/'

posts_df = pd.read_csv(REDDIT_S3_PATH + 'reddit_posts_4M.csv')
predictions_df = pd.read_csv(REDDIT_S3_PATH + 'test_predictions.csv')

posts_df['date'] = posts_df['created_utc'].apply(datetime.fromtimestamp)

In [ ]:
SEED = 42
MODEL_NAME = "distilbert-base-uncased"
LOG_DIR = "./news-classification"    
NUM_EPOCHS = 4                         
BATCH_SIZE = 40                        
MAX_SEQ_LENGTH = 256                   
LEARN_RATE = 5e-5                      
ACCUM_STEPS = 4                        
SEED = 42                              
FP16_PARAMS = None

In [ ]:
text = pd.DataFrame({
    "text" : df.text,
    "label" : df.category  
})

train, val = train_test_split(
    text, test_size=0.30, random_state=SEED)

train.reset_index(drop=True, inplace=True)
val.reset_index(drop=True, inplace=True)

val, test = train_test_split(
    val, test_size=0.10, random_state=SEED)

val.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

test_true = pd.DataFrame({
    "label" : test.label
})

test.drop(columns=["label"], axis=1, inplace=True)

In [ ]:
class TextClassificationDataset(Dataset):

    def __init__(self,
                 texts: List[str],
                 labels: List[str] = None,
                 label_dict: Mapping[str, int] = None,
                 max_seq_length: int = 512,
                 model_name: str = 'distilbert-base-uncased'):

        self.texts = texts
        self.labels = labels
        self.label_dict = label_dict
        self.max_seq_length = max_seq_length

        if self.label_dict is None and labels is not None:
            self.label_dict = dict(zip(sorted(set(labels)),
                                       range(len(set(labels)))))

        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        logging.getLogger(
            "transformers.tokenization_utils").setLevel(logging.FATAL)

        self.sep_vid = self.tokenizer.vocab["[SEP]"]
        self.cls_vid = self.tokenizer.vocab["[CLS]"]
        self.pad_vid = self.tokenizer.vocab["[PAD]"]

    def __len__(self):

        return len(self.texts)

    def __getitem__(self, index) -> Mapping[str, torch.Tensor]:

        x = self.texts[index]
        x_encoded = self.tokenizer.encode(
            x,
            add_special_tokens=True,
            max_length=self.max_seq_length,
            return_tensors="pt",
        ).squeeze(0)

        true_seq_length = x_encoded.size(0)
        pad_size = self.max_seq_length - true_seq_length
        pad_ids = torch.Tensor([self.pad_vid] * pad_size).long()
        x_tensor = torch.cat((x_encoded, pad_ids))

        mask = torch.ones_like(x_encoded, dtype=torch.int8)
        mask_pad = torch.zeros_like(pad_ids, dtype=torch.int8)
        mask = torch.cat((mask, mask_pad))

        output_dict = {
            "features": x_tensor,
            'attention_mask': mask
        }

        if self.labels is not None:
            y = self.labels[index]
            y_encoded = torch.Tensor(
                [self.label_dict.get(y, -1)]
            ).long().squeeze(0)
            output_dict["targets"] = y_encoded

        return output_dict

train_dataset = TextClassificationDataset(
    texts=train['text'].values.tolist(),
    labels=train['label'].values.tolist(),
    label_dict=None,
    max_seq_length=MAX_SEQ_LENGTH,
    model_name=MODEL_NAME
)

valid_dataset = TextClassificationDataset(
    texts=val['text'].values.tolist(),
    labels=val['label'].values.tolist(),
    label_dict=train_dataset.label_dict,
    max_seq_length=MAX_SEQ_LENGTH,
    model_name=MODEL_NAME
)

test_dataset = TextClassificationDataset(
    texts=test['text'].values.tolist(),
    labels=None,
    label_dict=None,
    max_seq_length=MAX_SEQ_LENGTH,
    model_name=MODEL_NAME
)

In [ ]:
import json

json.dumps({v:k for k,v in train_dataset.label_dict.items()})

In [ ]:
NUM_CLASSES = len(train_dataset.label_dict)
train_val_loaders = {
    "train": DataLoader(dataset=train_dataset,
                        batch_size=BATCH_SIZE, 
                        shuffle=True),
    "valid": DataLoader(dataset=valid_dataset,
                        batch_size=BATCH_SIZE, 
                        shuffle=False)    
}


class DistilBertForSequenceClassification(nn.Module):

    def __init__(self, pretrained_model_name: str, num_classes: int = None):

        super().__init__()

        config = AutoConfig.from_pretrained(
            pretrained_model_name, num_labels=num_classes)

        self.distilbert = AutoModel.from_pretrained(pretrained_model_name,
                                                    config=config)
        self.pre_classifier = nn.Linear(config.dim, config.dim)
        self.classifier = nn.Linear(config.dim, num_classes)
        self.dropout = nn.Dropout(config.seq_classif_dropout)

    def forward(self, features, attention_mask=None, head_mask=None):

        assert attention_mask is not None, "attention mask is none"
        distilbert_output = self.distilbert(input_ids=features,
                                            attention_mask=attention_mask,
                                            head_mask=head_mask)

        hidden_state = distilbert_output[0]  
        pooled_output = hidden_state[:, 0] 
        pooled_output = self.pre_classifier(pooled_output)  
        pooled_output = nn.ReLU()(pooled_output)  
        pooled_output = self.dropout(pooled_output)  
        logits = self.classifier(pooled_output)  

        return logits

model = DistilBertForSequenceClassification(pretrained_model_name=MODEL_NAME,
                                            num_classes=NUM_CLASSES)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARN_RATE)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

In [ ]:
NUM_CLASSES

In [ ]:
PATH="./export_model/state_dict"
torch.save(model.state_dict(), PATH)

In [ ]:
torch.load("news-classification/checkpoints/best_full.pth")

## Train

In [ ]:
!ls news-classification/checkpoints

In [ ]:
PATH="./export_model/export"
torch.save(model, PATH)


In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0"   
set_global_seed(SEED)                       
prepare_cudnn(deterministic=True)

In [ ]:
%%time 
runner = SupervisedRunner(input_key=("features","attention_mask"))

runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=train_val_loaders,
    callbacks=[
        AccuracyCallback(num_classes=NUM_CLASSES),
#       F1ScoreCallback(activation='Softmax'), # Tried it, but got an error on tensor shape
        OptimizerCallback(accumulation_steps=ACCUM_STEPS)
    ],
    fp16=FP16_PARAMS,
    logdir=LOG_DIR,
    num_epochs=NUM_EPOCHS,
    verbose=True,
    #resume=LOG_DIR
)

In [ ]:




import boto3

sm = boto3.client('sagemaker')
sm.stop_notebook_instance(NotebookInstanceName='string')

In [ ]:
print("done")

In [ ]:
!cp news-classification/checkpoints/train.3.pth "/gdrive/My Drive/"

In [ ]:
test_loaders = {
    "test": DataLoader(dataset=test_dataset,
                        batch_size=BATCH_SIZE, 
                        shuffle=False) 
}

runner.infer(
    model=model,
    loaders=test_loaders,
    callbacks=[
        CheckpointCallback(
            resume=f"{LOG_DIR}/checkpoints/best.pth"
        ),
        InferCallback(),
    ],   
    verbose=True
)

In [ ]:
predicted_probs = runner.state.callbacks[0].predictions['logits']
test_pred = pd.DataFrame(
    {
        "label": predicted_probs.argmax(axis=1)
    }
)
test_pred["label"] = test_pred["label"].map({v:k for k, v in train_dataset.label_dict.items()})

unique_label = train["label"].unique()
cmtx = pd.DataFrame(
    confusion_matrix(test_true, test_pred, labels=unique_label), 
    index=['{:}'.format(x) for x in unique_label], 
    columns=['{:}'.format(x) for x in unique_label]
)

plt.figure(figsize=(15, 10))
ax = sns.heatmap(cmtx, annot=True, fmt="d", cmap="YlGnBu")
ax.set_ylabel('Predicted')
ax.set_xlabel('Target');
ax.set_title("Figure 9 - Test Set Confusion Matrix", size=12)

acc = accuracy_score(test_true, test_pred)
prec = precision_score(test_true, test_pred, average="weighted")
f1 = f1_score(test_true, test_pred, average="weighted")
print(f"Test Accuracy: {acc}")
print(f"Test Precision: {prec}")
print(f"Test F1 Score: {f1}");

In [ ]:
!pip install hackernews-client -q

## Serve

In [ ]:
from hackernews import hn
news_client = hn.NewsClient()

def get_comm(x):
  try:
    c_id = x.kids[0]
    return requests.get(f"https://hacker-news.firebaseio.com/v0/item/{c_id}.json?print=pretty").json()['text']
  except:
    return ""

h = news_client.get_best_story(fetchMax=200)

headline_and_comment  =  [x.title + " " + get_comm(x) for x in h]


In [ ]:
test_dataset = TextClassificationDataset(
    texts=headline_and_comment,
    labels=None,
    label_dict=None,
    max_seq_length=MAX_SEQ_LENGTH,
    model_name=MODEL_NAME
)

test_loaders = {
    "test": DataLoader(dataset=test_dataset,
                        batch_size=BATCH_SIZE, 
                        shuffle=False) 
}

runner.infer(
    model=model,
    loaders=test_loaders,
    callbacks=[
        CheckpointCallback(
            resume=f"{LOG_DIR}/checkpoints/best.pth"
        ),
        InferCallback(),
    ],   
    verbose=True
)
predicted_probs = runner.state.callbacks[0].predictions['logits']
test_pred = pd.DataFrame(
    {
        "headline" : headline_and_comment,
        "label": pd.Series(np.argmax(predicted_probs,axis=1)).map({v:k for k, v in train_dataset.label_dict.items()})
    }
)

test_pred


In [ ]:
test_pred.query("label=='BUSINESS'").head(20)

In [ ]:
test_pred.tail(20)